In [2]:
import config_readwrite as crw
import glob
import numpy as np
import os, sys
import pandas as pd
import pybedtools as pbt

from scipy import stats

In [3]:
# biomarin MPRA
config, cfn= crw.read("/wynton/home/ahituv/fongsl/biomarin/bin/config.neuron.mpra.orig.ini")
MPRA = config["lib2"]['delta.mpra_centeredz'] #["enh_bed"] #["sig-wilcoxon"] reps l2(rna/dna)
ENH_BED = config["lib2"]['enh_bed']#["sig-wilcoxon"] reps l2(rna/dna)

MPRA_BED_NONA = ENH_BED.strip(".bed") + ".genomic_only.bed"
MPRA_BED_x_SONG = ENH_BED.strip(".bed") + ".x.SONG2020.bed"

MPRA_DATAPATH = "/wynton/group/ahituv/fongsl/projects/biomarin/data"
config["lib2"]['enh_bed_genomic']=MPRA_BED_NONA

SUPPORT = config["lib2"]["support"] # neighboring tile support

# Song data
DATA_PATH = "/wynton/group/ahituv/data/encode/shen_bdbag_2019/data/"
os.chdir(DATA_PATH) # go to plac-seq path

# write
combined = os.path.join(DATA_PATH, f"iN.eN.IPC.RG.HiC.bed")  # combine all hic coordinates together
collapsed = os.path.join(DATA_PATH, f"iN.eN.IPC.RG.HiC.collapsed.bed")  # collapse all hic annotation 
collapsed_cl = os.path.join(DATA_PATH, f"iN.eN.IPC.RG.HiC.collapsed.celltype.only.bed")  # collapse all hic annotation 

SONG_ATAC_PATH = "/wynton/group/ahituv/data/encode/shen_bdbag_2019/data/embargo/shen/accessible_chromatin/processed/peaks"
EN_ATAC = os.path.join(SONG_ATAC_PATH, "eN.ATAC-seq.narrowPeak")
IN_ATAC = os.path.join(SONG_ATAC_PATH, "iN.ATAC-seq.narrowPeak")

MPRA_BED_x_SONGEN = os.path.join(MPRA_DATAPATH,"eN.ATAC-seq.narrowPeak" + ".x.MPRA.bed")
MPRA_BED_x_SONGIN = os.path.join(MPRA_DATAPATH,"iN.ATAC-seq.narrowPeak" + ".x.MPRA.bed")

section = "song_2020_placseq"
#crw.check(section, config)

config[section]["path"] = DATA_PATH
config[section]["combined"] = combined
config[section]["collapsed"] = collapsed
config[section]["collapsed_cl"] = collapsed_cl
config[section]["mpra_bed_x_song"] = MPRA_BED_x_SONG
config[section]["atac_path"] = SONG_ATAC_PATH
config[section]["atac_in"] = IN_ATAC
config[section]["atac_en"] = EN_ATAC
config[section]["mpra_bed_x_songinATAC"] = MPRA_BED_x_SONGIN
config[section]["mpra_bed_x_songenATAC"] = MPRA_BED_x_SONGEN



crw.write(config, cfn)

# Main

## load plac

In [4]:
plac = pd.read_csv(combined, sep='\t')
plac

/scratch/fongsl/ipykernel_1463090/2542846049.py:1: DtypeWarning: Columns (1,2,3,4,5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  plac = pd.read_csv(combined, sep='\t')


,#chr,start,end,count,expected,fdr,celltype,id,link,link_id
0,chr1,890000,895000,36,13.7267118749079,7.08770821896245e-05,iN,iN.14,chr1:890000-895000_chr1:900000-905000,1
1,chr1,900000,905000,36,13.7267118749079,7.08770821896245e-05,iN,iN.14,chr1:890000-895000_chr1:900000-905000,2
2,chr1,1000000,1005000,13,3.17293864858796,0.0005259819865644,iN,iN.259,chr1:1000000-1005000_chr1:1300000-1305000,1
3,chr1,1005000,1010000,13,3.19541920865153,0.0015132910675641,iN,iN.1938,chr1:1005000-1010000_chr1:1305000-1310000,1
4,chr1,1020000,1025000,17,3.87774128536577,2.41664334727065e-05,iN,iN.218,chr1:1020000-1025000_chr1:1320000-1325000,1
...,...,...,...,...,...,...,...,...,...,...
226782,chr9,137450000,137455000,13,2.981913,0.000476,RG,RG.17410,chr9:137290000-137295000_chr9:137450000-137455000,2
226783,chr9,137465000,137470000,17,3.509233,0.000011,RG,RG.17351,chr9:137290000-137295000_chr9:137465000-137470000,2
226784,chr9,137535000,137540000,14,2.401291,0.000013,RG,RG.15806,chr9:137045000-137050000_chr9:137535000-137540000,2
226785,chr9,137560000,137565000,47,18.546754,0.000003,RG,RG.15813,chr9:137560000-137565000_chr9:137575000-137580000,1


In [5]:
### drop any na columns (positive controls, negative controls, synthetics, shuffled)

lib = pd.read_csv(ENH_BED, sep='\t')

print(lib.shape, lib.dropna().shape)
list(set(lib.loc[lib["enh.name"].isna(), "name"]))[:3]

(44044, 11) (42366, 11)


['Negative-chr18:60134410-60134679:-',
 'Backgroundseq2chr1:244100624-244100893:+|Pos1:115,Pos2:155|Motiffamily:glutmotif2,Motiffamily:glutmotif2|Motif1:TGAGATCTTAAC|Motif2:TTCCTTATATGT',
 'Positive-GLUT_chr10:26004717-26004986:+']

In [6]:
lib.loc[lib["enh.name"]=="enh.1530"]

,#chr,start.tile,end.tile,coor,#chr.enh,start.enh,end.enh,enh.id,enh.name,tile.order,name
37062,chr6,109381579.0,109381849.0,chr6:109381579-109381849,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,0.0,chr6:109381579-109381849
37063,chr6,109381599.0,109381869.0,chr6:109381599-109381869,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,1.0,chr6:109381599-109381869
37064,chr6,109381619.0,109381889.0,chr6:109381619-109381889,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,2.0,chr6:109381619-109381889
37065,chr6,109381639.0,109381909.0,chr6:109381639-109381909,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,3.0,chr6:109381639-109381909
37066,chr6,109381659.0,109381929.0,chr6:109381659-109381929,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,4.0,chr6:109381659-109381929
37067,chr6,109381679.0,109381949.0,chr6:109381679-109381949,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,5.0,chr6:109381679-109381949
37068,chr6,109381699.0,109381969.0,chr6:109381699-109381969,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,6.0,chr6:109381699-109381969
37069,chr6,109381719.0,109381989.0,chr6:109381719-109381989,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,7.0,chr6:109381719-109381989
37070,chr6,109381739.0,109382009.0,chr6:109381739-109382009,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,8.0,chr6:109381739-109382009


In [7]:
# if os.path.exists(MPRA_BED_NONA) is False:
genome_only = lib.loc[~lib["enh.name"].isna()].copy()
genome_only[genome_only.columns[1:3]] = genome_only[genome_only.columns[1:3]].astype(int)
genome_only.to_csv(MPRA_BED_NONA, sep='\t', index=False)  # write

## intersect mpra x song_2020

In [8]:
mprabed = pbt.BedTool(MPRA_BED_NONA)

#@if os.path.exists(MPRA_BED_x_SONG) is False:
mprabed = mprabed.sort()
song2020 = pbt.BedTool(collapsed_cl).sort()

mprabed.intersect(song2020, wa=True, wb=True, output=MPRA_BED_x_SONG)


song2020_atac_in = pbt.BedTool(IN_ATAC).sort()
song2020_atac_en = pbt.BedTool(EN_ATAC).sort()

mprabed.intersect(song2020_atac_en, wa=True, wb=True, output=MPRA_BED_x_SONGEN)

mprabed.intersect(song2020_atac_in, wa=True, wb=True, output=MPRA_BED_x_SONGIN)


<BedTool(/wynton/group/ahituv/fongsl/projects/biomarin/data/iN.ATAC-seq.narrowPeak.x.MPRA.bed)>

# load song intersection only

## atac peaks

In [18]:
cols = ['#chr',
        'start.tile',
        'end.tile',
        'coor',
        '#chr.enh',
        'start.enh',
        'end.enh',
        'enh.id',
        'enh.name',
        'tile.order',
        'name', "#chr_song", "start_song", "end_song", "peak"]
mpra_song_en = pd.read_csv(MPRA_BED_x_SONGEN, sep='\t', header=None, usecols=np.arange(0, 15))
mpra_song_en.columns=cols # columns
mpra_song_en["peak_eN"]="eN." + mpra_song_en["peak"] # anotate peak
mpra_song_en.loc[mpra_song_en["enh.name"]=="enh.1530"]

,#chr,start.tile,end.tile,coor,#chr.enh,start.enh,end.enh,enh.id,enh.name,tile.order,name,#chr_song,start_song,end_song,peak,peak_eN
22158,chr6,109381699,109381969,chr6:109381699-109381969,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,6.0,chr6:109381699-109381969,chr6,109381967,109383505,Peak_1349,eN.Peak_1349
22159,chr6,109381719,109381989,chr6:109381719-109381989,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,7.0,chr6:109381719-109381989,chr6,109381967,109383505,Peak_1349,eN.Peak_1349
22160,chr6,109381739,109382009,chr6:109381739-109382009,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,8.0,chr6:109381739-109382009,chr6,109381967,109383505,Peak_1349,eN.Peak_1349


In [19]:
mpra_song_in = pd.read_csv(MPRA_BED_x_SONGIN, sep='\t', header=None, usecols=np.arange(0, 15))
mpra_song_in.columns=cols
mpra_song_in["peak_iN"]="iN." + mpra_song_in["peak"]
mpra_song_in.loc[mpra_song_in["enh.name"]=="enh.1530"]

,#chr,start.tile,end.tile,coor,#chr.enh,start.enh,end.enh,enh.id,enh.name,tile.order,name,#chr_song,start_song,end_song,peak,peak_iN
19434,chr6,109381719,109381989,chr6:109381719-109381989,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,7.0,chr6:109381719-109381989,chr6,109381972,109383471,Peak_1804,iN.Peak_1804
19435,chr6,109381739,109382009,chr6:109381739-109382009,chr6,109381579.0,109382009.0,chr6:109381579-109382009,enh.1530,8.0,chr6:109381739-109382009,chr6,109381972,109383471,Peak_1804,iN.Peak_1804


## hic

In [11]:
cols = ['#chr',
        'start.tile',
        'end.tile',
        'coor',
        '#chr.enh',
        'start.enh',
        'end.enh',
        'enh.id',
        'enh.name',
        'tile.order',
        'name', "#chr_song", "start_song", "end_song", "celltype"]

mpra_song = pd.read_csv(MPRA_BED_x_SONG, sep='\t', header=None, names=cols)
mpra_song_en = pd.read_csv(MPRA_BED_x_SONGEN, sep='\t', header=None, names=cols)
mpra_song.head()

,#chr,start.tile,end.tile,coor,#chr.enh,start.enh,end.enh,enh.id,enh.name,tile.order,name,#chr_song,start_song,end_song,celltype
0,chr1,2440188,2440458,chr1:2440188-2440458,chr1,2440188.0,2440764.0,chr1:2440188-2440764,enh.3,0.0,chr1:2440188-2440458,chr1,2440000,2445000,eN
1,chr1,2440208,2440478,chr1:2440208-2440478,chr1,2440188.0,2440764.0,chr1:2440188-2440764,enh.3,1.0,chr1:2440208-2440478,chr1,2440000,2445000,eN
2,chr1,2440228,2440498,chr1:2440228-2440498,chr1,2440188.0,2440764.0,chr1:2440188-2440764,enh.3,2.0,chr1:2440228-2440498,chr1,2440000,2445000,eN
3,chr1,2440248,2440518,chr1:2440248-2440518,chr1,2440188.0,2440764.0,chr1:2440188-2440764,enh.3,3.0,chr1:2440248-2440518,chr1,2440000,2445000,eN
4,chr1,2440259,2440529,chr1:2440259-2440529,chr1,2440188.0,2440764.0,chr1:2440188-2440764,enh.3,4.0,chr1:2440259-2440529,chr1,2440000,2445000,eN


## MPRA data

In [13]:
mpra = pd.read_csv(MPRA, sep='\t')

mpra = pd.merge(mpra, lib[['enh.name', 'tile.order', 'name']], how="left").drop_duplicates()  # no shuffles, positive controls, neg controls 
mpra.head()

,name,gaba-label,glut-label,sig,celltype_dif,gaba,glut,delta.gaba-glut,fdr,-log10p_fdr,bs,enh.name,tile.order
0,"Backgroundseq172hr_top_98|Pos1:115,Pos2:155|Mo...",silencing,False,False,True,-2.699699,-1.696544,-1.003154,0.119390,0.923032,False,NaN,NaN
1,"Backgroundseq172hr_top_98|Pos1:115,Pos2:155|Mo...",silencing,False,False,True,-2.683608,-1.740263,-0.943346,0.493535,0.306682,False,NaN,NaN
2,"Backgroundseq172hr_top_98|Pos1:115,Pos2:155|Mo...",False,False,False,False,-2.228061,-1.842682,-0.385379,0.728743,0.137425,False,NaN,NaN
3,"Backgroundseq172hr_top_98|Pos1:115,Pos2:155|Mo...",silencing,False,False,True,-2.666574,-1.554678,-1.111896,0.364764,0.437989,False,NaN,NaN
4,"Backgroundseq172hr_top_98|Pos1:115,Pos2:155|Mo...",silencing,silencing,False,False,-2.756752,-2.505154,-0.251599,0.801537,0.096076,False,NaN,NaN


## support data

In [14]:
support = pd.read_csv(SUPPORT, sep='\t')

# enh_ids with neighbor support
supported_enhid = support.loc[support["neighbor_either_label"]!=False, "enh.name"].unique()
print(len(supported_enhid))

# merge support with mpra 
mpra = pd.merge(mpra, support, how="left").drop_duplicates()

support

1740


,name,tile.order,support_dif,support_dif_code,support_dif_name,neighbor_dif,neighbor_gaba_label,neighbor_glut_label,enh.name,neighbor_either_label
0,chr15:58748908-58749178,0.0,0,0,NaN,NaN,NaN,NaN,enh.566,None.None
1,chr15:58748928-58749198,1.0,0,0,NaN,NaN,NaN,NaN,enh.566,None.None
2,chr15:58748948-58749218,2.0,0,0,NaN,NaN,NaN,NaN,enh.566,None.None
3,chr15:58748968-58749238,3.0,0,0,NaN,NaN,NaN,NaN,enh.566,None.None
4,chr15:58748988-58749258,4.0,0,0,NaN,NaN,NaN,NaN,enh.566,None.None
...,...,...,...,...,...,...,...,...,...,...
42171,chr19:44891616-44891886,134.0,0,0,NaN,NaN,NaN,NaN,enh.810,None.None
42172,chr19:44891636-44891906,135.0,0,0,NaN,NaN,NaN,NaN,enh.810,None.None
42173,chr19:44891656-44891926,136.0,0,0,NaN,NaN,NaN,NaN,enh.810,None.None
42174,chr19:44891676-44891946,137.0,1,10,cat-only,NaN,NaN,NaN,enh.810,None.None


In [15]:
len(set(mpra["enh.name"]))

1699

## merge MPRA w hic

In [23]:
# add PLAC
int = pd.merge(mpra, mpra_song, how="left").drop_duplicates()

# add eN ATAC
int = pd.merge(
    int, mpra_song_en[["enh.name", "peak_eN"]], how="left").drop_duplicates()

# add iN ATAC
int = pd.merge(
    int, mpra_song_in[["enh.name", "peak_iN"]], how="left").drop_duplicates()

drop_ind = int.loc[(int["name"].str.contains("shuf")) |
                   (int["name"].str.contains("seq")) |
                   (int["name"].str.contains("top")) |
                   (int["name"].str.contains("bottom"))
                   ].index


int = int.loc[~int.index.isin(drop_ind)]
int.head()

,name,gaba-label,glut-label,sig,celltype_dif,gaba,glut,delta.gaba-glut,fdr,-log10p_fdr,...,#chr.enh,start.enh,end.enh,enh.id,#chr_song,start_song,end_song,celltype,peak_eN,peak_iN
974,Negative-GABA_chr10:26471819-26472088:+,False,False,False,False,-1.614119,-1.590097,-0.024022,0.924293,0.034190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
975,Negative-GABA_chr10:26798964-26799233:-,False,False,False,False,-1.785743,-1.543796,-0.241947,0.138986,0.857028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
976,Negative-GABA_chr10:26799114-26799383:-,False,False,False,False,-1.425355,-1.231964,-0.193391,0.662463,0.178838,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
977,Negative-GABA_chr10:26799129-26799398:-,False,False,False,False,-1.444640,-1.449871,0.005231,0.985000,0.006564,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
978,Negative-GABA_chr10:26799159-26799428:-,False,False,False,False,-1.590640,-1.124827,-0.465813,0.384536,0.415063,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
len(lib["enh.id"].unique()), len(int["enh.id"].unique())

(1741, 602)

In [25]:
cols = ['gaba-label', 'glut-label', "enh.name"]
int[cols].drop_duplicates().groupby(cols[:-1])[cols[-1]].count()

gaba-label  glut-label
False       False         1670
            activating     960
            silencing      123
activating  False          287
            activating     284
silencing   False           99
            activating       1
            silencing      105
Name: enh.name, dtype: int64

# count MPRA enh.loci

## remove shuf, pos, synthetic MPRA elemetns

In [29]:
drop_ind = mpra.loc[(mpra["name"].str.contains("shuf")) |
                   (mpra["name"].str.contains("seq")) |
                   (mpra["name"].str.contains("top")) |
                   (mpra["name"].str.contains("bottom"))
                   ].index

mpra = mpra.loc[~mpra.index.isin(drop_ind)]

## count MPRA enhancer loci

In [33]:
cols = ['gaba-label', 'glut-label', "enh.name"]
mpra_count = mpra[cols].drop_duplicates().dropna()

In [36]:
# get sets of gaba, glut active enhancers
gaba_set = set(mpra_count.loc[mpra_count["gaba-label"]!="False", "enh.name"])
glut_set = set(mpra_count.loc[mpra_count["glut-label"]!="False", "enh.name"])

In [50]:
# common loci
common_loci = (gaba_set.intersection(glut_set))
print("N loci w/ gaba and glut activity, overall", len(common_loci))

N loci w/ gaba and glut activity, overall 484


In [46]:
# gaba-only loci
gaba_loci = (gaba_set.difference(glut_set))
print("N loci w/ gaba activity, no glut", len(gaba_loci))

N loci w/ gaba activity, no glut 75


In [47]:
# glut only-loci
glut_loci = (glut_set.difference(gaba_set))
print("N loci w/ glut activity, no gaba", len(glut_loci))

N loci w/ glut activity, no gaba 607


## commonly active loci - subregional analysis

In [52]:
common_mpra = mpra.loc[mpra["enh.name"].isin(common_loci)]

In [80]:
common_w_dif_subregions = set(common_mpra.loc[common_mpra["celltype_dif"]==True, "enh.name"])
print("N loci w/ glut and gaba, but subregions with cell-specific activity", len(common_w_dif_subregions))

N loci w/ glut and gaba, but subregions with cell-specific activity 458


In [79]:
common_w_nodif_subregions = set(common_mpra.loc[
    (common_mpra["celltype_dif"] == False), 
    "enh.name"]
)
len(common_w_nodif_subregions)

484

In [85]:
common_only= common_w_nodif_subregions.difference(common_w_dif_subregions)

In [90]:
mpra_count.loc[mpra_count["enh.name"].isin(common_only)].groupby(["gaba-label", "glut-label"])["enh.name"].count()

gaba-label  glut-label
False       False         25
activating  activating     9
silencing   silencing     17
Name: enh.name, dtype: int64

In [93]:
print("N loci w/ glut and gaba shared activity ONLY", len(common_only))

N loci w/ glut and gaba shared activity ONLY 26


# FET - active element in song cell type? 

    # test 1 - MPRA active enhancer enriched for plac-seq?
                active either | not active either
        Song+
    not Song+

    # test 2 - MPRA active tile label enriched for plac-seq?
                active either tile label | not active either tile label
        Song+
    not Song+

    # test 3 - MPRA enriched for cell-type-specific plac-seq (e.g. glut v. en | gaba v. in)
                active glut | not active glut
        en+
    not en+

In [95]:
def fetPlac(active_mpra, inactive_mpra, inplac, outplac):
    # intersect elements to get counts. 
    a = len(active_mpra.intersection(inplac))
    b = len(inactive_mpra.intersection(inplac))
    c = len(active_mpra.intersection(outplac))
    d = len(inactive_mpra.intersection(outplac))
    
    contingency_table=[[a,b], [c,d]]
    
    print(contingency_table)
    OR, p = stats.fisher_exact(contingency_table)
    print(OR, p)

## Test 1 - is MPRA active enhancer (one active tile) enriched for PLAC-seq?
 - use tile-level inference                                     
    

In [98]:
active_mpra = set(int.loc[(int["gaba-label"] != "False") |
                          (int["glut-label"] != "False"), 
                  mpra_unit])

1167

In [99]:
412+189

601

In [96]:
mpra_unit = "enh.name"

# get active MPRA enhancer loci (wo neighbor support, tile support only)
active_mpra = set(int.loc[(int["gaba-label"] != "False") |
                          (int["glut-label"] != "False"), 
                  mpra_unit])

# get inactive MPRA enhancer loci (wo neighbor support, tile support only)
inactive_mpra_ = set(int.loc[(int["gaba-label"] == "False") &
                             (int["glut-label"] == "False"), mpra_unit])
# remove any elements that have both active and inactive regions
inactive_mpra = inactive_mpra_.difference(active_mpra)

# get all enhancers that overlap song elements
inplac = set(int.loc[~int["#chr_song"].isna(), mpra_unit])

# get all enhancers that don't overlap song elements
outplac_ = set(int.loc[int["#chr_song"].isna(), mpra_unit])

# remove any elements that are both in and out of plac
outplac = outplac_.difference(inplac)

fetPlac(active_mpra, inactive_mpra, inplac, outplac)

[[412, 189], [755, 341]]
0.9845614772767091 0.91280620581736


### Results test 2 - No. enrichment of MPRA active loci (one-tile definition) with Plac-seq

    # test 2 - MPRA active enhancer (one-tile active in either glut or gaba) enriched for plac-seq?
                active either | not active either
        Song+          A            B
    not Song+          C            D

    [[A,B], [C,D]]
    [[412, 189], [755, 341]]

    FET OR, P
       0.9845614772767091 0.91280620581736



## test 3 - cell-type-active elements enriched in cell-type plac?

In [102]:
cls = [("gaba", "iN"), 
      ("glut", "eN")]
mpra_unit = "enh.name"  # measure at the locus level or the tile level ("name")


### Results test3 - cell-type-active elements are not enriched for cell-type plac overlap
gaba, iN-containing annotations
     
      test 3 - MPRA enriched for cell-type-specific plac-seq (e.g. glut v. en | gaba v. in)
                active glut/gaba | not active glut/gaba
        en/in+
    not en/in+



     gaba, plac
    [[60, 541], [121, 975]]
    0.8936618750095476 0.5121526626263884
    
    glut, plac
    [[225, 376], [354, 742]]
    1.2542823656689506 0.03677774332185711
    
    gaba, iN-containing annotations
    [[32, 276], [149, 1241]]
    0.0 0.06905449642466382

    glut, eN-containing annotations
    [[127, 232], [452, 887]]
    0.8391903914590747 0.6043975831507908

In [103]:
for mpra_cl , plac_cl in cls:
    print(mpra_cl , plac_cl)
    # get active MPRA enhancer loci (w neighbor support)
    active_mpra = set(int.loc[int[f"{mpra_cl}-label"]!="False", mpra_unit])
    
    # get inactive MPRA enhancer loci (w neighbor support)
    inactive_mpra_ = set(int.loc[int[f"{mpra_cl}-label"]=="False", mpra_unit])
    
    # remove any elements that have both active and inactive regions in an enhancer locus from inactive enhancer set. 
    inactive_mpra = inactive_mpra_.difference(active_mpra)
    
    # get all enhancers that overlap song elements
    int["celltype"] = int["celltype"].fillna("none")

    # # TEST1 ALL PLAC
    # get all enhancers that overlap song elements
    inplac = set(int.loc[~int["#chr_song"].isna(), mpra_unit])
    
    # get all enhancers that don't overlap song elements
    outplac_ = set(int.loc[int["#chr_song"].isna(), mpra_unit])
    # remove any elements that are both in and out of plac
    outplac = outplac_.difference(inplac)

    # # TEST2 ONLYCL-matching PLAC
    inplac_cl = set(int.loc[int["celltype"].str.contains(plac_cl), mpra_unit])
    
    # get all enhancers that do not overlap song elements
    outplac_cl_ = set(int.loc[(~int["celltype"].str.contains(plac_cl)), mpra_unit])
    # remove any elements that are both in and out of plac_cl
    outplac_cl = outplac_cl_.difference(inplac_cl)

    # run FET
    fetPlac(active_mpra, inactive_mpra, inplac, outplac)

    fetPlac(active_mpra, inactive_mpra, inplac_cl, outplac_cl_)

gaba iN
[[204, 397], [356, 740]]
1.0681232841819261 0.5529161124811286
[[109, 199], [463, 945]]
1.1179547847227498 0.42336638328063564
glut eN
[[388, 213], [704, 392]]
1.0142979086641057 0.9156476282193515
[[234, 125], [876, 488]]
1.0428493150684932 0.7570040684625096


## differences in activity

In [104]:
mpra.head()

,name,gaba-label,glut-label,sig,celltype_dif,gaba,glut,delta.gaba-glut,fdr,-log10p_fdr,bs,enh.name,tile.order,support_dif,support_dif_code,support_dif_name,neighbor_dif,neighbor_gaba_label,neighbor_glut_label,neighbor_either_label
974,Negative-GABA_chr10:26471819-26472088:+,False,False,False,False,-1.614119,-1.590097,-0.024022,0.924293,0.034190,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
975,Negative-GABA_chr10:26798964-26799233:-,False,False,False,False,-1.785743,-1.543796,-0.241947,0.138986,0.857028,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
976,Negative-GABA_chr10:26799114-26799383:-,False,False,False,False,-1.425355,-1.231964,-0.193391,0.662463,0.178838,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
977,Negative-GABA_chr10:26799129-26799398:-,False,False,False,False,-1.444640,-1.449871,0.005231,0.985000,0.006564,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
978,Negative-GABA_chr10:26799159-26799428:-,False,False,False,False,-1.590640,-1.124827,-0.465813,0.384536,0.415063,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
cls = [("celltype_dif", "iN"), 
      ("celltype_dif", "eN")]
mpra_unit = "enh.name"  # measure at the locus level or the tile level ("name")

for mpra_cl , plac_cl in cls:
    print(mpra_cl , plac_cl)
   # get active MPRA enhancer loci (w neighbor support)
    active_mpra = set(int.loc[int[mpra_cl]!=False, mpra_unit])
    
    # get inactive MPRA enhancer loci (w neighbor support)
    inactive_mpra_ = set(int.loc[int[mpra_cl]==False, mpra_unit])
    
    # remove any elements that have both active and inactive regions in an enhancer locus from inactive enhancer set. 
    inactive_mpra = inactive_mpra_.difference(active_mpra)
    
    # get all enhancers that overlap song elements
    int["celltype"] = int["celltype"].fillna("none")

    # # TEST ALL PLAC
    # get all enhancers that overlap song elements
    inplac = set(int.loc[~int["#chr_song"].isna(), mpra_unit])
    
    # get all enhancers that don't overlap song elements
    outplac_ = set(int.loc[int["#chr_song"].isna(), mpra_unit])
    # remove any elements that are both in and out of plac
    outplac = outplac_.difference(inplac)

    # # TEST CL-matching PLAC
    #inplac_cl = set(int.loc[int["celltype"]==plac_cl, mpra_unit])
    
    # get all enhancers that do not overlap song elements
    outplac_cl_ = set(int.loc[int["celltype"]!=plac_cl, mpra_unit])
    # remove any elements that are both in and out of plac_cl
    outplac_cl = outplac_cl_.difference(inplac_cl)

    # run FET
    fetPlac(active_mpra, inactive_mpra, inplac, outplac)

    fetPlac(active_mpra, inactive_mpra, inplac_cl, outplac_cl_)

celltype_dif iN
[[406, 195], [735, 361]]
1.0226129426129427 0.8711615712341801
[[240, 119], [1126, 545]]
0.9761631117811246 0.8525803749015346
celltype_dif eN
[[406, 195], [735, 361]]
1.0226129426129427 0.8711615712341801
[[240, 119], [1106, 534]]
0.9737565912440925 0.8522052255922918


## ATAC peaks
- are active MPRA elements enriched in exictatory or inhibitory atac peaks? 

In [107]:
cls = [("gaba", "peak_iN"), 
      ("glut", "peak_eN")]
mpra_unit = "enh.name"  # measure at the locus level or the tile level ("name")

for mpra_cl , plac_cl in cls:
    print(mpra_cl , plac_cl)
    # get active MPRA enhancer loci (w neighbor support)
    active_mpra = set(int.loc[int[f"{mpra_cl}-label"]!="False", mpra_unit])
    
    # get inactive MPRA enhancer loci (w neighbor support)
    inactive_mpra_ = set(int.loc[int[f"{mpra_cl}-label"]=="False", mpra_unit])
    
    # remove any elements that have both active and inactive regions in an enhancer locus from inactive enhancer set. 
    inactive_mpra = inactive_mpra_.difference(active_mpra)
    
    # get all enhancers that overlap song elements
    #int["celltype"] = int["celltype"].fillna("none")

    # # TEST ALL PLAC
    # get all enhancers that overlap song elements
    inplac = set(int.loc[~int[plac_cl].isna(), mpra_unit])
    
    # get all enhancers that don't overlap song elements
    outplac_ = set(int.loc[int[plac_cl].isna(), mpra_unit])
    # remove any elements that are both in and out of plac
    outplac = outplac_.difference(inplac)


    # run FET
    fetPlac(active_mpra, inactive_mpra, inplac, outplac)

    

gaba peak_iN
[[298, 581], [262, 556]]
1.0884629028655517 0.43843179522229336
glut peak_eN
[[633, 323], [459, 282]]
1.2040308383415286 0.07373398402181922


In [58]:
# annotate differentially labeled peaks

peaks = int[["enh.name", "peak_eN", "peak_iN"]].drop_duplicates().copy().dropna(how="all")

peaks['peak_eN_'] = peaks['peak_eN'].apply(lambda x: -1 if "Peak" in str(x) else 0) # -1 is Glut
peaks['peak_iN_'] = peaks['peak_iN'].apply(lambda x: 1 if "Peak" in str(x) else 0)  # 1 is Gaba
peaks["peak_dif"] = peaks['peak_iN_']+ peaks['peak_eN_']  # difference is -1 for glut-specific, 0 for not specific, 1 for gaba specific
peaks["peak_sum"] = peaks['peak_iN_']+ peaks['peak_eN_'].abs()  # if 2, peak is accessible in both cells. 
peaks.sort_values(by="peak_dif")

,enh.name,peak_eN,peak_iN,peak_eN_,peak_iN_,peak_dif,peak_sum
566364,enh.1100,eN.Peak_51127,NaN,-1,0,-1,1
356641,enh.595,eN.Peak_35218,NaN,-1,0,-1,1
356605,enh.612,eN.Peak_123092,NaN,-1,0,-1,1
355956,enh.591,eN.Peak_25433,NaN,-1,0,-1,1
355937,enh.590,eN.Peak_25433,NaN,-1,0,-1,1
...,...,...,...,...,...,...,...
708739,enh.1332,NaN,iN.Peak_109567,0,1,1,1
1052632,enh.1721,NaN,iN.Peak_35593,0,1,1,1
905739,enh.1524,NaN,iN.Peak_139956,0,1,1,1
1052524,enh.1720,NaN,iN.Peak_88786,0,1,1,1


In [59]:
int = pd.merge(int, peaks, how="left")

# enrichment of gaba-specific active elements in GABA-specific peaks? 
- no significant enrichment. 

In [62]:
cls = [("gaba", "peak_iN_", "glut"), 
      ("glut", "peak_eN_", 'gaba')]
mpra_unit = "enh.name"  # measure at the locus level or the tile level ("name")

for mpra_cl , plac_cl, mpra_not_cl in cls:
    print(mpra_cl , plac_cl)
   
    # get active MPRA enhancer loci (w neighbor support)
    active_mpra = set(int.loc[(int[f"neighbor_{mpra_cl}_label"].astype(str)!="nan")&
                              (int[f"neighbor_{mpra_not_cl}_label"].astype(str)=="nan"), 
                              mpra_unit])
    
    # get inactive MPRA enhancer loci (w neighbor support)
    inactive_mpra_ = set(int.loc[(int[f"neighbor_{mpra_cl}_label"].astype(str)!="nan")&
                                 (int[f"neighbor_{mpra_not_cl}_label"].astype(str)=="nan"), 
                                mpra_unit])
    
    # remove any elements that have both active and inactive regions in an enhancer locus from inactive enhancer set. 
    inactive_mpra = inactive_mpra_.difference(active_mpra)
    
    # get all enhancers that overlap song elements
    #int["celltype"] = int["celltype"].fillna("none")

    # # TEST ALL PLAC
    # get all enhancers that overlap song elements
    inplac = set(int.loc[(int[plac_cl]!=0)&
                 (int["peak_sum"]==1), mpra_unit])
    
    # get all enhancers that don't overlap song elements
    outplac_ = set(int.loc[(int[plac_cl]==0)&
                 (int["peak_sum"]==1), mpra_unit])
    # remove any elements that are both in and out of plac
    outplac = outplac_.difference(inplac)


    # run FET
    fetPlac(active_mpra, inactive_mpra, inplac, outplac)

    

gaba peak_iN_
[[10, 0], [24, 0]]
nan 1.0
glut peak_eN_
[[58, 0], [34, 0]]
nan 1.0


## BM candidates

In [108]:
int[["celltype", "enh.name"]].drop_duplicates().groupby("celltype").count()

,enh.name
celltype,
IPC,23
"IPC,RG",36
"IPC,RG,eN",34
"IPC,RG,eN,iN",159
"IPC,RG,iN",28
"IPC,eN",17
"IPC,eN,iN",16
"IPC,iN",11
RG,100


In [124]:
info_cols = [#'name',
    "enh.name", 
 'gaba-label',
 'glut-label',
 'sig',
 'celltype_dif',
 '-log10p_fdr',
 'bs',
 'tile.order',
    '#chr_song',
 'start_song',
 'end_song',
 'celltype', "peak_eN", 'peak_iN']

In [125]:
# enh123 #slc2a1 
# enh 1530 SYNGAP1, excitatory neuron
enhs=["1157", "1159",  "43", "57", # gaba candidates: 57 and 1159 are inhibitory peaks, 1157 is an excitatory peak, 43 is no peaks 
        "1531", "1603", '1628' # glut candidates: 1531 has no peaks, 1603 is excitatory and inhibitory, 1628 is excitatory
     ]
for e in enhs:
    loc = int.loc[(int["enh.name"]==(f"enh.{e}")), info_cols]
    #if list(set(loc["celltype"]))[0]!="none":
    print(f"enh.{e}\n\n", loc, "\n\n")

enh.1157

         enh.name gaba-label glut-label    sig  celltype_dif  -log10p_fdr  \
688285  enh.1157      False      False  False         False     1.255028   
688286  enh.1157      False      False  False         False     0.806970   
688287  enh.1157      False      False  False         False     0.998218   
688288  enh.1157      False      False   True         False     1.460266   
688289  enh.1157      False      False  False         False     0.808038   
688290  enh.1157      False      False  False         False     0.665558   
688291  enh.1157      False      False  False         False     0.985447   
688292  enh.1157      False      False  False         False     0.952187   
688293  enh.1157      False      False  False         False     1.161807   

           bs  tile.order #chr_song  start_song  end_song celltype  \
688285  False         0.0       NaN         NaN       NaN     none   
688286  False         1.0       NaN         NaN       NaN     none   
688287  False     

In [123]:
loc

,enh.name,gaba-label,glut-label,sig,celltype_dif,-log10p_fdr,bs,tile.order,#chr_song,start_song,end_song,celltype
1012384,enh.1628,False,False,False,False,1.209525,False,0.0,NaN,NaN,NaN,none
1012385,enh.1628,False,False,False,False,0.998719,False,1.0,NaN,NaN,NaN,none
1012386,enh.1628,False,False,False,False,0.867382,False,2.0,NaN,NaN,NaN,none
1012387,enh.1628,False,False,False,False,0.407158,False,3.0,NaN,NaN,NaN,none
1012388,enh.1628,False,False,False,False,0.078064,False,4.0,NaN,NaN,NaN,none
1012389,enh.1628,False,False,False,False,0.084777,False,5.0,NaN,NaN,NaN,none
1012390,enh.1628,False,False,True,False,1.605409,False,6.0,NaN,NaN,NaN,none
1012391,enh.1628,False,False,False,False,0.564087,False,7.0,NaN,NaN,NaN,none
1012392,enh.1628,False,False,False,False,0.191603,False,8.0,NaN,NaN,NaN,none
1012393,enh.1628,False,False,True,False,1.619496,False,9.0,NaN,NaN,NaN,none


In [35]:
#enh 1530
plac.loc[(plac["start"]=='109380000') & (plac["#chr"]=='chr6')]

,#chr,start,end,count,expected,fdr,celltype,id,link,link_id
147608,chr6,109380000,109385000,12,3.82557551796205,0.003183063515626,eN,eN.9621,chr6:108560000-108565000_chr6:109380000-109385000,2
147609,chr6,109380000,109385000,17,3.44369076410255,1.65726015668907e-06,eN,eN.9644,chr6:108555000-108560000_chr6:109380000-109385000,2


In [118]:
#enh 123
plac.loc[(plac["start"]=='180500000') & (plac["#chr"]=='chr1')]

,#chr,start,end,count,expected,fdr,celltype,id,link,link_id
3364,chr1,180500000,180505000,13,1.93387637632898,1.43863048159754e-05,iN,iN.124,chr1:180500000-180505000_chr1:181180000-181185000,1
3365,chr1,180500000,180505000,13,2.34282177008648,9.41871892109445e-05,iN,iN.127,chr1:180500000-180505000_chr1:181110000-181115000,1
3366,chr1,180500000,180505000,16,3.42122822979022,2.13850882081469e-05,iN,iN.237,chr1:180500000-180505000_chr1:181090000-181095000,1
3367,chr1,180500000,180505000,12,3.54914031577137,0.0043810422789134,iN,iN.352,chr1:180500000-180505000_chr1:181085000-181090000,1
3368,chr1,180500000,180505000,23,2.85325782946977,3.04188093703056e-11,iN,iN.894,chr1:180500000-180505000_chr1:180855000-180860000,1
3369,chr1,180500000,180505000,26,3.08712901861514,3.76774643257841e-13,iN,iN.1115,chr1:180500000-180505000_chr1:180880000-180885000,1
3370,chr1,180500000,180505000,16,2.34149569361732,6.79827926112344e-07,iN,iN.2119,chr1:180500000-180505000_chr1:180860000-180865000,1
3371,chr1,180500000,180505000,14,2.35923226312276,2.13541842775166e-05,iN,iN.2140,chr1:180500000-180505000_chr1:180875000-180880000,1
49205,chr1,180500000,180505000,13,2.10253062583694,2.71841388494469e-05,IPC,IPC.78,chr1:180500000-180505000_chr1:181150000-181155000,1
49206,chr1,180500000,180505000,13,2.39503850372902,9.61410695392654e-05,IPC,IPC.1305,chr1:180500000-180505000_chr1:180830000-180835000,1
